In [49]:
import pandas as pd
import numpy as np

#### 1. Data import

In [50]:
conflict_df = pd.read_csv('datasets/ucdp-prio-acd-171.csv')
conflict_df.head()

,conflictid,location,sidea,sidea2nd,side b,sidebid,sideb2nd,incomp,terr,year,...,epend,ependdate,ependprec,gwnoa,gwnoa2nd,gwnob,gwnob2nd,gwnoloc,region,version
0,200,Bolivia,Government of Bolivia,NaN,Popular Revolutionary Movement,719,NaN,2,NaN,1946,...,1,1946-07-21,1.0,145,NaN,NaN,NaN,145,5,17.1
1,200,Bolivia,Government of Bolivia,NaN,MNR,720,NaN,2,NaN,1952,...,1,1952-04-12,1.0,145,NaN,NaN,NaN,145,5,17.1
2,200,Bolivia,Government of Bolivia,NaN,ELN,721,NaN,2,NaN,1967,...,1,1967-10-16,1.0,145,NaN,NaN,NaN,145,5,17.1
3,201,Cambodia (Kampuchea),Government of France,NaN,Khmer Issarak,160,NaN,1,Cambodia,1946,...,0,NaN,NaN,220,NaN,NaN,NaN,811,3,17.1
4,201,Cambodia (Kampuchea),Government of France,NaN,Khmer Issarak,160,NaN,1,Cambodia,1947,...,0,NaN,NaN,220,NaN,NaN,NaN,811,3,17.1


#### 2. Dataframe process

- remove useless columns
- remove all "Government of..."
- remove text between parantheses
- complete/correct "terr" columns

In [51]:
# Remove useless columns
columns_to_drop = ['conflictid', 'sidebid', 'incomp', 'terr', 'year', 'startprec', 'sidea2nd',
                   'startprec2', 'epend', 'ependprec', 'gwnoa', 'gwnoa2nd', 'sideb2nd',
                   'gwnob', 'gwnob2nd', 'gwnoloc', 'version']

conflict_df.drop(columns_to_drop, axis=1, inplace=True)

# Remove columns
conflict_df.replace({'Government of ': ''}, regex=True, inplace=True)
# Remove text between parentheses 
conflict_df.location.replace({r'\([^()]+\)' : ''}, regex=True, inplace=True)

conflict_df.head()

,location,sidea,side b,intensity,cumint,type,startdate,startdate2,ependdate,region
0,Bolivia,Bolivia,Popular Revolutionary Movement,2,1,3,1946-07-18,1946-07-21,1946-07-21,5
1,Bolivia,Bolivia,MNR,1,1,3,1946-07-18,1952-04-09,1952-04-12,5
2,Bolivia,Bolivia,ELN,1,1,3,1946-07-18,1967-03-31,1967-10-16,5
3,Cambodia,France,Khmer Issarak,1,0,1,1946-08-31,1946-08-31,NaN,3
4,Cambodia,France,Khmer Issarak,1,0,1,1946-08-31,1946-08-31,NaN,3


In [52]:
columns = ("location", "sidea", "side b", "intensity", "cumint", "type", 
           "startdate", "startdate2", "ependdate", "region")

conflict_df_tmp = pd.DataFrame(columns=columns)

for index in range(len(conflict_df)):

    locations = conflict_df.loc[index, 'location'].split(",")
    regions = conflict_df.loc[index, 'region'].split(",")
    
    if len(regions) > 1:
        
        pairs = [(locations[i], regions[i]) for i in range(len(locations))]

        for i, pair in enumerate(pairs):

            conflict_row_df = conflict_df.loc[index:index,]
            
            if i > 0: 
                conflict_row_df.set_value(index, 'location', pair[0][1:])
                conflict_row_df.set_value(index, 'region', pair[1])
                #conflict_row_df.loc[index, 'location'] = pair[0][1:]
                #conflict_row_df.loc[index, 'region']   = pair[1]
            else: 
                conflict_row_df.set_value(index, 'location', pair[0])
                conflict_row_df.set_value(index, 'region', pair[1])
                
            # Add the new line to the df
            conflict_df_tmp = conflict_df_tmp.append(conflict_row_df)
        
        # Remove initial line
        conflict_df.drop(index, inplace=True)#.reset_index(drop=True)
                  
    elif len(locations) > 1:
        conflict_df.loc[index, 'location'] = locations[0]
                
            
#test      
conflict_df = pd.concat([conflict_df,conflict_df_tmp]).reset_index(drop=True)

In [53]:
conflict_df 

,location,sidea,side b,intensity,cumint,type,startdate,startdate2,ependdate,region
0,Bolivia,Bolivia,Popular Revolutionary Movement,2,1,3,1946-07-18,1946-07-21,1946-07-21,5
1,Bolivia,Bolivia,MNR,1,1,3,1946-07-18,1952-04-09,1952-04-12,5
2,Bolivia,Bolivia,ELN,1,1,3,1946-07-18,1967-03-31,1967-10-16,5
3,Cambodia,France,Khmer Issarak,1,0,1,1946-08-31,1946-08-31,NaN,3
4,Cambodia,France,Khmer Issarak,1,0,1,1946-08-31,1946-08-31,NaN,3
5,Cambodia,France,Khmer Issarak,1,0,1,1946-08-31,1946-08-31,NaN,3
6,Cambodia,France,Khmer Issarak,1,0,1,1946-08-31,1946-08-31,NaN,3
7,Cambodia,France,Khmer Issarak,1,0,1,1946-08-31,1946-08-31,NaN,3
8,Cambodia,France,Khmer Issarak,1,0,1,1946-08-31,1946-08-31,NaN,3
9,Cambodia,France,Khmer Issarak,1,0,1,1946-08-31,1946-08-31,NaN,3


#### 3. Keep ex-colonies


In [ ]:
colonized_df = pd.read_csv('datasets/colonies_wikipedia.csv')
colonized_countries = list(colonized_df["Colonized Country"])

In [ ]:
conflict_df_tmp = conflit_df.head(100)

for index in range(len(conflict_df_tmp)):
    
    if conflict_df_tmp["location"][index]
    print(conflict_df_tmp["location"][index])

In [ ]:
conflict_df_tmp["location"][0] in colonized_countries